In [1]:
from mlband.imports import *
import mlband.data
import mlband.config
import mlband.model
import mlband.features

In [2]:
data_set_path = 'data/mp-test-2/'
results_dir = 'results/test-2/'

print('Preparing the config...', flush=True)
config = mlband.config.Config(
    data_path=data_set_path, 
    workers=4, 
    results_dir=results_dir,
    original_features=True,
    print_freq=1000,
    )

Preparing the config...


In [3]:
print('Downloading the data from MP...' ,flush=True)
df, data = mlband.data.get_list_of_materials(num_chunks=40)

Retrieving SummaryDoc documents:   0%|          | 0/40000 [00:00<?, ?it/s]

Number of missing values:  2


In [4]:
print('Creating the dataset...', flush=True)
mlband.data.create_dataset(df=df, path=data_set_path)

Creating the dataset...


/Users/ali/mambaforge/envs/cgcnn/lib/python3.11/site-packages/pymatgen/core/periodic_table.py:120: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


In [5]:
print('Creating the physical features...', flush=True)
mlband.features.one_hot_encode_elements(config)

Creating the physical features...


In [6]:
%%time
print('Creating and training the model...', flush=True)
mlband.model.train(config)

Creating and training the model...
Training the model...
Epoch: [0][0/500]	Time 11.146 (11.146)	Data 8.056 (8.056)	Loss 1.0336 (1.0336)	MAE 0.899 (0.899)
